<a href="https://colab.research.google.com/github/Srijani-coder/Learning_ML_Path/blob/main/Cleaning_Tweet_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [61]:
import re
import json

# Read the input JSON file
with open('/content/sample_data/10000 tweets 1.json', 'r', encoding='utf-8') as file:
    data = file.read()

if '/* 3830 */' in data:
  print(True)

# Define a regular expression pattern to match /* x */ and line gaps
pattern = r'/*\s+\d+\s+\*/\s*\n*'

# Split the data into tweet dictionaries using the pattern as a delimiter
tweets = re.split(pattern, data)

tweets[0]

True


'/* 10000 Tweets */\n\n\n/*'

In [62]:
tweets[1]

'{\n    "_id" : ObjectId("abe415aa5e7133a4a61c0d8317875"),\n    "id" : "149715690143449899009",\n    "objectType" : "activity",\n    "actor" : {\n        "objectType" : "person",\n        "id" : "10243188921458",\n        "link" : "http://www.twitter.com/losebabyweight1",\n        "displayName" : "losebabyweight",\n        "postedTime" : "2010-09-09T22:40:10.000Z",\n        "image" : "https://pbs.twimg.com/profile_images/522696965503455233/WfF2aJ7N_normal.png",\n        "summary" : "http://www.losebabyweight.com.au offers mums safe and proven plans to lose weight. Lose an average of 1kg a week.",\n        "links" : [ \n            {\n                "href" : "http://www.losebabyweight.com.au",\n                "rel" : "me"\n            }\n        ],\n        "friendsCount" : 218,\n        "followersCount" : 1960,\n        "listedCount" : 17,\n        "statusesCount" : 14439,\n        "twitterTimeZone" : "Sydney",\n        "verified" : false,\n        "utcOffset" : "39600",\n        "pr

In [63]:
len(tweets)

10001

In [64]:
import re
from datetime import datetime

def extract_cleaned_tweet(input_string):
    # Define regular expressions for extracting relevant data
    id_str_pattern = r'"id" : "([^"]+)"'
    text_pattern = r'"text" : "([^"]+)"'
    actor_id_pattern = r'"actor" : \{[^}]*"id" : "([^"]+)"'
    display_name_pattern = r'"displayName" : "([^"]+)"'
    preferred_username_pattern = r'"preferredUsername" : "([^"]+)"'
    location_pattern = r'"location" : \{[^}]*"displayName" : "([^"]+)"'
    link_pattern = r'"link" : "([^"]+)"'
    summary_pattern = r'"summary" : "([^"]+)"'
    posted_time_pattern = r'"postedTime" : "([^"]+)"'
    expanded_url_pattern = r'"expanded_url" : "([^"]+)"'

    # Extract data using regular expressions
    id_str_match = re.search(id_str_pattern, input_string)
    text_match = re.search(text_pattern, input_string)
    actor_id_match = re.search(actor_id_pattern, input_string)
    display_name_match = re.search(display_name_pattern, input_string)
    preferred_username_match = re.search(preferred_username_pattern, input_string)
    location_match = re.search(location_pattern, input_string)
    if location_match is None:
      location_match = '[]'
    else:
      location_match = location_match.group(1)
    link_match = re.search(link_pattern, input_string)
    summary_match = re.search(summary_pattern, input_string)
    posted_time_match = re.search(posted_time_pattern, input_string)
    expanded_url_match = re.search(expanded_url_pattern, input_string)

    # Create the cleaned tweet dictionary
    cleaned_tweet = {
        "created_at": datetime.strptime(posted_time_match.group(1), "%Y-%m-%dT%H:%M:%S.000Z").strftime("%a %b %d %H:%M:%S +0000 %Y"),
        "id_str": id_str_match.group(1),
        "text": text_match.group(1),
        "user": {
            "id": int(actor_id_match.group(1)),
            "name": display_name_match.group(1),
            "screen_name": preferred_username_match.group(1),
            "location": location_match,
            "url": link_match.group(1),
            "description": summary_match.group(1)
        },
        "place": {},
        "entities": {
            "hashtags": [],
            "urls": [],
            "user_mentions": []
        }
    }

    # Extract hashtags and URLs
    # Generate a regex pattern to extract hashtags
    hashtags_pattern = r'"twitter_entities"\s*:\s*\{(?:\s*"hashtags"\s*:\s*\[\s*\{[^}]*"text"\s*:\s*"([^"]+)"[^}]*\]\s*)?\}'
    # Find hashtags using the pattern
    hashtags_match = re.search(hashtags_pattern, input_string)

    # Initialize a list to store extracted hashtags
    extracted_hashtags = []

    # Check if "twitter_entities" and "hashtags" exist and are not empty
    if hashtags_match and hashtags_match.group(1):
      extracted_hashtags = hashtags_match.group(1).split('","')

    urls = re.findall(expanded_url_pattern, input_string)

    for hashtag in extracted_hashtags:
        cleaned_tweet["entities"]["hashtags"].append(hashtag)

    for url in urls:
        cleaned_tweet["entities"]["urls"].append({
            "url": url,
            "unwound": {
                "url": url,
                "title": ""
            }
        })

    return cleaned_tweet

# Example usage:



In [65]:
cleaned_tweet_list = []
error_indices = []  # To store the indices of items that raise exceptions
i = 1

while i <= 10000:
    try:
        cl_twt = extract_cleaned_tweet(tweets[i])
        cleaned_tweet_list.append(cl_twt)
    except Exception as e:
        error_indices.append(i)  # Add the index to the error_indices list
        print(f"Error at index {i}: {str(e)}")
    finally:
        i = i + 1

# Print the indices of items that raised exceptions
print("Indices of items with errors:", error_indices)






Indices of items with errors: []


In [66]:
len(error_indices)

0

In [67]:
len(cleaned_tweet_list)

10000

In [75]:
with open('cleaned_tweet.json', 'w+') as file:
  file.write('[')
  i = 0
  while(i < 10000):
    file.write(str(cleaned_tweet_list[i]) + ",\n")
    i = i + 1
  file.write(']')